In [1]:
from scipy.io import wavfile
import numpy as np
import scipy
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
# from IPython.display import Audio 
# from vscode_audio import Audio
import IPython.display
import librosa
import librosa.display
import re

def myAudio(audio: np.ndarray, sr: int):
    """
    Use instead of IPython.display.Audio as a workaround for VS Code.
    `audio` is an array with shape (channels, samples) or just (samples,) for mono.
    """

    if np.ndim(audio) == 1:
        channels = [audio.tolist()]
    else:
        channels = audio.tolist()

    return IPython.display.HTML("""
        <script>
            if (!window.audioContext) {
                window.audioContext = new AudioContext();
                window.playAudio = function(audioChannels, sr) {
                    const buffer = audioContext.createBuffer(audioChannels.length, audioChannels[0].length, sr);
                    for (let [channel, data] of audioChannels.entries()) {
                        buffer.copyToChannel(Float32Array.from(data), channel);
                    }
            
                    const source = audioContext.createBufferSource();
                    source.buffer = buffer;
                    source.connect(audioContext.destination);
                    source.start();
                }
            }
        </script>
        <button onclick="playAudio(%s, %s)">Play</button>
    """ % (json.dumps(channels), sr))

In [2]:
import os
meta = pd.DataFrame(os.listdir('audio/')).set_axis(['wavfile'], axis=1)
cat = [None]*meta.size
for i in range(meta.size):
    cat[i] = re.split("_", re.sub("_", ":", meta.wavfile[i], 1))[0]
meta['label'] = cat

In [3]:

choice=['flute:synthetic','bass:electronic', 'vocal:acoustic']

In [4]:
train = pd.DataFrame()
for i in range(3):
    c = choice[i]
    train = pd.concat([train, meta.query("label == @c").sample(1000)], ignore_index=True)
train

,wavfile,label
0,flute_synthetic_004-054-100.wav,flute:synthetic
1,flute_synthetic_002-082-127.wav,flute:synthetic
2,flute_synthetic_002-022-050.wav,flute:synthetic
3,flute_synthetic_002-034-075.wav,flute:synthetic
4,flute_synthetic_004-083-075.wav,flute:synthetic
...,...,...
2995,vocal_acoustic_022-081-127.wav,vocal:acoustic
2996,vocal_acoustic_005-061-050.wav,vocal:acoustic
2997,vocal_acoustic_023-077-100.wav,vocal:acoustic
2998,vocal_acoustic_024-064-050.wav,vocal:acoustic


In [5]:
X = [None]*train.wavfile.size
y = train.label

def scaledNNfeatures(filename):
    data, rate = librosa.load(filename)
    return np.mean(librosa.feature.mfcc(y=data, sr=rate, n_mfcc=40).T, axis=0)

for i in range(train.wavfile.size):
    X[i] = scaledNNfeatures('audio/'+train.wavfile[i])
    X[i] = X[i].reshape(40)
    y[i] = train.label[i]



## Train-Test Split

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
model=logreg.fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

Accuracy of Logistic regression classifier on training set: 0.82
Accuracy of Logistic regression classifier on test set: 0.78


C:\Users\lyric\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
mixed_mfcc=pd.read_csv('features1.csv')

In [9]:
mixed_mfcc_1=mixed_mfcc[mixed_mfcc.columns[0:40]]

In [10]:
probs=model.predict_proba(mixed_mfcc_1)

In [11]:
cnt=0
for i in range (150):
        if(probs[i][0]<probs[i][1] and probs[i][0]<probs[i][2]):
            x=2;
            if (x==mixed_mfcc.at[i,'label']):
                cnt=cnt+1
        if(probs[i][1]<probs[i][0] and probs[i][1]<probs[i][2]):
            x=3;
            if (x==mixed_mfcc.at[i,'label']):
                cnt=cnt+1
        if(probs[i][2]<probs[i][1] and probs[i][2]<probs[i][0]):
            x=1;
            if (x==mixed_mfcc.at[i,'label']):
                cnt=cnt+1
            
            
        

In [14]:
accuracy=cnt/mixed_mfcc.shape[0]

In [15]:
accuracy

0.6333333333333333